In [7]:
import sys
sys.path.append('../')

from src.llm.llm_interface import LLM
import os
from openai import OpenAI
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from trulens_eval import Tru
from dotenv import load_dotenv, find_dotenv
envs = load_dotenv(find_dotenv(), override=True)

In [8]:
tru = Tru()

In [20]:
from trulens_eval.tru_custom_app import instrument
from trulens_eval import TruCustomApp, Feedback
from trulens_eval.feedback import Groundedness, GroundTruthAgreement

In [12]:
client = OpenAI()

class APP:
    def __init__(self, aoai_client: OpenAI):
        self.client = aoai_client
        
    @instrument
    def completion(self, prompt):
        completion = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                temperature=0,
                messages=
                [
                    {"role": "user",
                    "content": 
                    f"Please answer the question: {prompt}"
                    }
                ]
                ).choices[0].message.content
        return completion

In [30]:
llm = APP(client)

In [31]:
golden_set = [
    {"query": "who authored the Black Sun Rising?", "response": "Kazuo Ishiguro"},
    {"query": "Where did Alexander the Great originate from?", "response": "Macedonia"}
]

f_groundtruth = Feedback(GroundTruthAgreement(golden_set).agreement_measure, name = "Ground Truth").on_input_output()

✅ In Ground Truth, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ground Truth, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [32]:
tru_app = TruCustomApp(llm, app_id='LLM App v1', feedbacks=[f_groundtruth])

In [35]:
with tru_app as recording:
    llm.completion("who authored the Black Sun Rising?")
    llm.completion('Where did Alexander the Great originate from?')

In [36]:
tru.get_leaderboard(app_ids=[tru_app.app_id])

,Ground Truth,latency,total_cost
app_id,,,
LLM App v1,0.716667,0.125,0.000063


In [44]:
recording[0].__dict__

{'record_id': 'record_hash_8e863437a37a633eaaf98df15beb8f77',
 'app_id': 'LLM App v1',
 'cost': Cost(n_requests=1, n_successful_requests=1, n_classes=0, n_tokens=31, n_stream_chunks=0, n_prompt_tokens=19, n_completion_tokens=12, cost=5.25e-05),
 'perf': Perf(start_time=datetime.datetime(2024, 4, 6, 2, 36, 36, 857865), end_time=datetime.datetime(2024, 4, 6, 2, 36, 37, 459770)),
 'ts': datetime.datetime(2024, 4, 6, 2, 36, 37, 459976),
 'tags': '-',
 'meta': None,
 'main_input': 'who authored the Black Sun Rising?',
 'main_output': 'The Black Sun Rising was authored by C.S. Friedman.',
 'main_error': None,
 'calls': [RecordAppCall(stack=[RecordAppCallMethod(path=Lens().app, method=Method(obj=Obj(cls=__main__.APP, id=140379979779328, init_bindings=None), name='completion'))], args={'prompt': 'who authored the Black Sun Rising?'}, rets='The Black Sun Rising was authored by C.S. Friedman.', error=None, perf=Perf(start_time=datetime.datetime(2024, 4, 6, 2, 36, 36, 857865), end_time=datetime.d

In [2]:
university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

In [3]:
harvard = """
Harvard University is a private Ivy League research university in Cambridge, Massachusetts. Founded in 1636 as Harvard College and named for its first benefactor, Puritan clergyman John Harvard, it is the oldest institution of higher learning in the United States. Its influence, wealth, and rankings have made it one of the most prestigious universities in the world.
"""

In [4]:
oai_client = OpenAI()

In [5]:
embedding = oai_client.embeddings.create(model='text-embedding-ada-002', input=university_info)

In [6]:
float_array = embedding.data[0].embedding

In [7]:
embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'),
                                             model_name="text-embedding-ada-002")


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Universities",
                                                      embedding_function=embedding_function)

In [8]:
vector_store.add("uni_info", documents=university_info)

In [9]:
vector_store.add('harvard', documents=harvard)

In [11]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()

In [12]:
from trulens_eval import Huggingface

In [13]:
hf = Huggingface()

In [ ]:
hf.hallucination_evaluator(

In [11]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
        query_texts=query,
        n_results=2
    )
        return results['documents'][0]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = oai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=
        [
            {"role": "user",
            "content": 
            f"We have provided context information below. \n"
            f"---------------------\n"
            f"{context_str}"
            f"\n---------------------\n"
            f"Given this information, please answer the question: {query}"
            }
        ]
        ).choices[0].message.content
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

In [12]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI

import numpy as np

provider = OpenAI()

grounded = Groundedness(groundedness_provider=provider)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets.collect() .


In [13]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_groundedness, f_answer_relevance, f_context_relevance])

In [14]:
with tru_rag as recording:
    rag.query("When was the University of Washington founded?")

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
tru.get_leaderboard(app_ids=["RAG v1"])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
RAG v1,0.0,0.9,1.0,1.0,0.000316


In [16]:
tru.run_dashboard()

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.26:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
Groundedness(